In [50]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [190]:
from transformers import TFBertForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('Tolerblanc/klue-bert-finetuned')
model = TFBertForSequenceClassification.from_pretrained('Tolerblanc/klue-bert-finetuned', output_attentions=True)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--Tolerblanc--klue-bert-finetuned/snapshots/0bf8dc4eefb6ab60b25f02cf3568deb6f23652af/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--Tolerblanc--klue-bert-finetuned/snapshots/0bf8dc4eefb6ab60b25f02cf3568deb6f23652af/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--Tolerblanc--klue-bert-finetuned/snapshots/0bf8dc4eefb6ab60b25f02cf3568deb6f23652af/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Tolerblanc--klue-bert-finetuned/snapshots/0bf8dc4eefb6ab60b25f02cf3568deb6f23652af/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Tolerblanc--klue-bert-finetuned/snapshots/0bf8dc4eefb6ab60b25f02cf3568deb6f23652af/config.json
Model config BertConfig {
  "_name_

In [191]:
text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer, 
    model=model, 
    framework='tf',
    return_all_scores=True,
    device=0
)

## **마스킹하는 함수**

In [302]:
def hate_expression(text_classifier, tokenizer, model,text):
  output = text_classifier(text)[0]
  clean = output[0]['score']
  curse = output[1]['score']
  print(f'{text} 가 입력되었으며,')
  if clean > curse:
    print(f'모델은 이 문장을 {clean * 100}% 확률로 깨끗한 문장이라고 추론했습니다.')
    return text
  else:
    print(f'모델은 이 문장을 {curse * 100}% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.')
    #문장을 단어단위로 분류
    words = text.split()
    masked_words = []

    for word in words:
        #wordpice토큰화를 진행하고, 텍스트를 정수 배열 변환
        inputs = tokenizer.encode_plus(word, return_tensors="tf")
        
        #하나의 문장만 사용해서 [0]으로 표현
        outputs = model(inputs)[0] 

        #정규화시키고,  일반 표현과 혐오표현에 대한 각각의 확률값을 배열로 표현 
        probabilities = tf.nn.softmax(outputs, axis=-1)

        #혐오표현과 일반표현의 확률이 너무 낮은 경우 임계값 설정
        threshold = 0.07514262                      
        if probabilities.numpy()[0, 1] > threshold:
          label_predictions = 1
        else:
          #확률값이 높은 부분의 위치를 숫자로 표현(일반표현 0, 혐오표현 1)
          label_predictions = tf.argmax(probabilities, axis=1).numpy()[0]

        # 0: 일반, 1: 혐오
        if label_predictions ==1:
            masked_word = "*" * len(word)
            masked_words.append(masked_word)
        else:
            masked_words.append(word)

    masked_text = " ".join(masked_words)
    return masked_text

In [193]:
text = '저게 시대적언어면 한남충도 시대적언어 아니노 ㅋㅋ	'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

저게 시대적언어면 한남충도 시대적언어 아니노 ㅋㅋ	 가 입력되었으며,
모델은 이 문장을 99.55450296401978% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
저게 시대적언어면 **** 시대적언어 아니노 ㅋㅋ


In [194]:
text = '배고픈데 앞에서 만두 먹네 개시키'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

배고픈데 앞에서 만두 먹네 개시키 가 입력되었으며,
모델은 이 문장을 99.68923926353455% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
배고픈데 앞에서 만두 먹네 ***


In [195]:
text = 'ㅅㅂ'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

ㅅㅂ 가 입력되었으며,
모델은 이 문장을 88.87845277786255% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
**


In [196]:
text = '마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나 가 입력되었으며,
모델은 이 문장을 99.95373487472534% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
마 *** 아 몇평이고 맷개드갔노 니 ****** ***


In [181]:
text = '좌배 까는건 ㅇㅂ'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

좌배 까는건 ㅇㅂ 가 입력되었으며,
모델은 이 문장을 71.53672575950623% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
좌배 *** **


In [303]:
text = '오마쥬가 뭔진 앎?'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

오마쥬가 뭔진 앎? 가 입력되었으며,
모델은 이 문장을 99.93797540664673% 확률로 깨끗한 문장이라고 추론했습니다.
오마쥬가 뭔진 앎?


In [198]:
text='크로캅이 본야스키 개털던거 생각나네 ㅋㅋㅋㅋㅋㅋ진짜 개잡듯 잡아팼지 ㅋㅋㅋㅋ전직 은행원 본야스키 ㅋ'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

크로캅이 본야스키 개털던거 생각나네 ㅋㅋㅋㅋㅋㅋ진짜 개잡듯 잡아팼지 ㅋㅋㅋㅋ전직 은행원 본야스키 ㅋ 가 입력되었으며,
모델은 이 문장을 94.540935754776% 확률로 깨끗한 문장이라고 추론했습니다.
None


In [199]:
text='무게치는거에 인생버린새끼들'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

무게치는거에 인생버린새끼들 가 입력되었으며,
모델은 이 문장을 99.88462924957275% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
무게치는거에 *******


In [200]:
text='버러지들'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

버러지들 가 입력되었으며,
모델은 이 문장을 79.22764420509338% 확률로 깨끗한 문장이라고 추론했습니다.
None


In [201]:
text='창렬이노'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

창렬이노 가 입력되었으며,
모델은 이 문장을 99.96140599250793% 확률로 깨끗한 문장이라고 추론했습니다.
None


In [202]:
text='내가 세금 안내면 너 같은 동사무소에서 프린트나 하는 새끼는 월급 못받고 굶어 뒈져야 돼 병신새끼야.'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

내가 세금 안내면 너 같은 동사무소에서 프린트나 하는 새끼는 월급 못받고 굶어 뒈져야 돼 병신새끼야. 가 입력되었으며,
모델은 이 문장을 99.95182752609253% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
내가 세금 안내면 너 같은 동사무소에서 프린트나 하는 *** 월급 못받고 굶어 *** 돼 ******


In [203]:
text='강철멘탈이네...대한민국에 성매매 종사자가 수만 수십만인데 그걸 이용하는 남자는 수십배는 되지 않겟냐? 다 먹고 살고 있는거 보면 아닌척 하는 넘들이 가증스러운거지'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

강철멘탈이네...대한민국에 성매매 종사자가 수만 수십만인데 그걸 이용하는 남자는 수십배는 되지 않겟냐? 다 먹고 살고 있는거 보면 아닌척 하는 넘들이 가증스러운거지 가 입력되었으며,
모델은 이 문장을 94.25874352455139% 확률로 깨끗한 문장이라고 추론했습니다.
None


In [229]:
text='이ㅂㅅ은 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 버러지같은새키'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

이ㅂㅅ은 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 버러지같은새키 가 입력되었으며,
모델은 이 문장을 99.95836615562439% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
**** 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 *******


In [252]:
text='문빈은 너무 착하고 여려 혼자 맘고생 하다가 떠나고 패배는 정신승리 하면서 출소기념 소주병 퍼포먼스 이게 맞나'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

문빈은 너무 착하고 여려 혼자 맘고생 하다가 떠나고 패배는 정신승리 하면서 출소기념 소주병 퍼포먼스 이게 맞나 가 입력되었으며,
모델은 이 문장을 99.60590600967407% 확률로 깨끗한 문장이라고 추론했습니다.
None


In [289]:
text='나라 생긴지 몇 년 됐다고 저런 전통도 있었냐 ㅋㅋ ㅂㄹㅈ ㅋㅋㅋ'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

나라 생긴지 몇 년 됐다고 저런 전통도 있었냐 ㅋㅋ ㅂㄹㅈ ㅋㅋㅋ 가 입력되었으며,
모델은 이 문장을 88.39884996414185% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
나라 *** 몇 * 됐다고 저런 전통도 있었냐 ㅋㅋ *** ㅋㅋㅋ


In [294]:
text='니넨얼마나잘살아왔냐ㅋㅋ 열등감오지네'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

니넨얼마나잘살아왔냐ㅋㅋ 열등감오지네 가 입력되었으며,
모델은 이 문장을 97.36399054527283% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
니넨얼마나잘살아왔냐ㅋㅋ ******


In [295]:
text='이새키보소'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

이새키보소 가 입력되었으며,
모델은 이 문장을 99.84443783760071% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
*****


In [296]:
text='쓰레기는 변하지 않아~'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

쓰레기는 변하지 않아~ 가 입력되었으며,
모델은 이 문장을 99.90399479866028% 확률로 깨끗한 문장이라고 추론했습니다.
None


In [298]:
text='요목조목 다 따져봐도 특출나게 잘난것도 없는 난쟁이놈이 전생에 나라라도 구했는지 ㅋㅋ 천운을 탔구나...  계속 깝치는 모습보니 안경벗기고 스크류 드라이버로 눈쑤셔버리고 싶다.'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)


요목조목 다 따져봐도 특출나게 잘난것도 없는 난쟁이놈이 전생에 나라라도 구했는지 ㅋㅋ 천운을 탔구나...  계속 깝치는 모습보니 안경벗기고 스크류 드라이버로 눈쑤셔버리고 싶다. 가 입력되었으며,
모델은 이 문장을 99.92628693580627% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
요목조목 다 따져봐도 특출나게 잘난것도 없는 ***** 전생에 **** 구했는지 ㅋㅋ 천운을 탔구나... 계속 *** 모습보니 안경벗기고 스크류 드라이버로 ****** 싶다.


In [299]:
text='이게 돈이면 다 되는 대한민국 클라쓰쥐~~~~~!!!!!'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

이게 돈이면 다 되는 대한민국 클라쓰쥐~~~~~!!!!! 가 입력되었으며,
모델은 이 문장을 55.15960454940796% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
이게 돈이면 다 되는 대한민국 클라쓰쥐~~~~~!!!!!
